# Importing and Querying Data with PostgreSQL (Northwind Database)

Describtion of the Northwind Database: https://en.wikiversity.org/wiki/Database_Examples/Northwind 

Example SQL-Queries based on the Northwind Database: https://www.w3schools.com/sql/default.asp

SQL Tutorials: https://www.youtube.com/watch?v=zpnHsWOy0RY&list=PLP9IO4UYNF0UQkBXlTMSw0CYsxv-GDkkI

## Frequently used SQL clauses (for in-depth learning)

<div>
  <style>
    .sql-keyword {
      color: blue;
    }
  </style>
  <span class="sql-keyword">SELECT</span><br>
  <span class="sql-keyword">SELECT DISTINCT</span><br>
  <span class="sql-keyword">FROM</span><br>
  <span class="sql-keyword">WHERE</span><br>
  <span class="sql-keyword">AND</span><br>
  <span class="sql-keyword">LIKE</span><br>
  <span class="sql-keyword">GROUP BY</span><br>
  <span class="sql-keyword">HAVING</span><br>
  <span class="sql-keyword">ORDER BY</span><br>
  <span class="sql-keyword">FULL JOIN</span><br>
  <span class="sql-keyword">INNER JOIN</span><br>
  <span class="sql-keyword">LEFT JOIN</span><br>
  <span class="sql-keyword">RIGHT JOIN</span><br>
  <span class="sql-keyword">ON</span><br>
  <span class="sql-keyword">LIMIT</span><br>
  <span class="sql-keyword">AS</span><br>
  <span class="sql-keyword">AVG</span><br>
  <span class="sql-keyword">COUNT</span><br>
  <span class="sql-keyword">MIN</span><br>
  <span class="sql-keyword">MAX</span><br>
  <span class="sql-keyword">SUM</span><br>
  <span class="sql-keyword">BETWEEN</span><br>
  <span class="sql-keyword">IN</span><br>
  <span class="sql-keyword">IS NULL</span><br>
  <span class="sql-keyword">IS NOT NULL</span><br> 
  <span class="sql-keyword">ROUND</span><br> 
  <span class="sql-keyword">CAST</span><br>
  <span class="sql-keyword">NUMERIC</span><br>
  <span class="sql-keyword">INTEGER</span>  
</div>

## Libraries and Settings

In [1]:
# Libraries
import os
import pandas as pd
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt

# Settings
import warnings
warnings.filterwarnings("ignore")

# Current working directory
print(os.getcwd())

/


## Read data from .csv files to separate data frames 

In [7]:
# Define path to data
data_path = '/workspace/Data'

# Read CSV files
csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]

# Dictionary to hold dataframes from CSV files
dfs = {}

# Read data from .csv files
for file in csv_files:
    file_path = os.path.join(data_path, file)
    df_name = os.path.splitext(file)[0]  # Use file name without extension as dataframe name
    df = pd.read_csv(file_path)
    df.columns = [col.lower() for col in df.columns]
    dfs[df_name] = df

# Read data from Excel file
excel_file_path = os.path.join(data_path, 'Northwind_Database.xlsx')
excel_dfs = pd.read_excel(excel_file_path, sheet_name=None)

# Convert all column names to lowercase in Excel DataFrames
for sheet_name, df in excel_dfs.items():
    df.columns = [col.lower() for col in df.columns]
    dfs[sheet_name] = df

# Check dimensions of DataFrames
for name, df in dfs.items():
    print(f"DataFrame: {name}, Shape: {df.shape}")

DataFrame: Shippers, Shape: (3, 3)
DataFrame: Products, Shape: (77, 6)
DataFrame: Employees, Shape: (10, 6)
DataFrame: Suppliers, Shape: (29, 8)
DataFrame: Categories, Shape: (8, 3)
DataFrame: Orders, Shape: (196, 5)
DataFrame: Customers, Shape: (91, 7)
DataFrame: OrderDetails, Shape: (518, 4)
DataFrame: apartments_data_prepared, Shape: (1008, 10)
DataFrame: ER-Diagram, Shape: (2, 2)


## Create database connection

In [9]:
# Set up database connection
user = "pgadmin"
password = "geheim"
host = "db"
port = "5432"
database = "postgres"

# Erstellen der Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test database connection
with engine.connect() as connection:
    result = connection.execute(text('SELECT current_database()'))
    print(result.fetchone())

# Dispose the engine
engine.dispose()

('postgres',)


## Write data frames to database tables

In [10]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Drop tables if they exist
with engine.connect() as connection:
    # List of tables to drop
    tables = ['categories', 'customers', 'employees', 'shippers', 'suppliers', 'products', 'orders', 'orderdetails']
    
    for table in tables:
        # Drop the table if it exists
        connection.execute(text(f"DROP TABLE IF EXISTS {table} CASCADE"))

# Write DataFrames to database
for table_name, df in dfs.items():
    # Write DataFrame to SQL table
    df.to_sql(table_name, engine, if_exists='replace', index=False)

# Dispose the engine
engine.dispose()

## List tables in the database

In [5]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Open a connection
with engine.connect() as connection:

    # Execute the query
    result = connection.execute(text("""SELECT table_name
                                        FROM information_schema.tables
                                        WHERE table_schema = 'public'"""))
    
    # Fetch and print the results
    for row in result:
        print(row[0])

# Dispose the engine
engine.dispose()

apartment_table
Shippers
Products
Employees
Suppliers
Categories
Orders
Customers
OrderDetails
apartments_data_prepared


## Create a SQL query to retrieve product information from a database.

In [6]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Write data to table
df_sub = pd.read_sql_query('''SQL Query comes here''', con=engine)
                          con=engine)

# Dispose the engine
engine.dispose()

# Show the data
df_sub

IndentationError: unexpected indent (3574155145.py, line 6)

## Plot product prices

In [ ]:
# Plot Histogram
fig = plt.figure( figsize=(7,4))
plt.xticks(fontsize=14, rotation=0)
plt.yticks(fontsize=14, rotation=0)
n, bins, patches = plt.hist(x=df_sub['price'], 
                            bins=20, 
                            color='#C53115',
                            alpha=1.00, 
                            rwidth=0.95
                   )
plt.grid(True)
plt.ticklabel_format(style='plain')
plt.grid(axis='y', alpha=0.75)

# Set labels
plt.xlabel('Price (USD)', fontsize=10, labelpad=10)
plt.ylabel('Frequency', fontsize=10, labelpad=10)
plt.title('Histogram of product prices', fontsize=12, pad=10)

# Set fontsize of tick labels
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)

plt.show()

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')